# Approach
本道题是21题的进阶版！   

方法一：分治法(Divide and Conquer)   
该方法基于21题：假设lists中有n个linked lists，我们将这n个linked lists分成两个一组，如果有奇数个linked list，我们就给那个单独的linked list分配None就好了(具体见代码，实现起来很简单)！一共可以得到n/2组！对每一组的两个linekd lists使用21题的mergeTwoLists函数，先将它们两个merge成一个sorted linked list，这样我们就得到了n/2个新的linked lists！重复上述操作直到只剩下一个linked list为止，这就是答案！这样比“前两个先合并，合并好了再跟第三个合并，然后第四个一直到第n个”效率高多了！O(logn) vs. O(n)！

方法二：Heap/Priority Queue   
该方法基于priority queue(min heap)：首先把n个linked list的第一个node都加入pq中，它们会自动排序。取出pq中val最小的那个node加入最终结果(linked list)中，然后把取出的node的下一个node加入pq中！然后继续从pq中取出val最小的node加入最终结果中，然后把取出的node的下一个node加入pq中！重复上述操作直到pq中没有node为止！此时最终结果的linked list就是n个linked list合并完的结果！   
该方法的精髓在于一开始把n个linked list的第一个node放入pq，其实就已经把所有的linked list存进去了！看似只有第一个node，但是由于linked list的特性，第一个node就可以代表整个linked list(可以通过next的方式遍历本linked list中的所有node)！   

# Note
'<' not supported between instances of 'ListNode' and 'ListNode'，这是因为我们直接把tuple:(node.val, node)放入pq中，python3的heapq要求我们加进pq的tuple里的每个元素都要可以比较！这样的话假如tuple的第一个元素相等，还能比较第二个元素作为排序的依据！但是我们根本就没有办法比较node！因此需要在代码中加入"ListNode.__lt__ = lambda self, other: self.val < other.val"，这样我们就可以根据node.val来比较node了！此时既然可以根据node.val比较node了，那我们直接往pq中加node即可！！！


# Code

In [ ]:
# Divide and Conquer
class Solution:
    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        # 为了不让lists[0] range out of index！
        if len(lists) == 0:
            return None
        # 分治法，不断对lists中的linked lists两两分组使用mergeTwoLists，直到只剩下一个linked list为止！
        while len(lists) > 1:
            # 用来存储每轮两两分组merge后新生成的sorted linked list，数量只有当前lists中linked lists的一半！
            # 每轮结束时要将其赋给lists！
            mergedList = []
            # 注意step=2！这是将相邻的两个lined list分成一组！
            for i in range(0, len(lists), 2):
                list1 = lists[i]
                # 假如lists中有奇数个linked lists，那么最后一个linked list没有配对的，直接分配一个None即可！
                list2 = lists[i + 1] if i + 1 < len(lists) else None
                mergedList.append(self.mergeTwoLists(list1, list2))
            # 更新lists，只有之前一半数量的sorted linked lists！这个新的lists要进入下次循环继续两两分组merge！
            lists = mergedList
        # 退出while loop后lists中只有一个linked list，即为最终答案！
        return lists[0]
    
    # 21题
    def mergeTwoLists(self, list1, list2):
        dummy_head = ListNode()
        curr, l1, l2 = dummy_head, list1, list2

        while l1 != None or l2 != None:
            val1 = l1.val if l1 else float('inf')
            val2 = l2.val if l2 else float('inf')

            if val1 < val2:
                curr.next = l1
                l1 = l1.next
            else:
                curr.next = l2
                l2 = l2.next
            curr = curr.next
        
        return dummy_head.next

In [ ]:
# Heap/Priority Queue   
class Solution:
    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        # 使得可以根据node.val来比较node！
        ListNode.__lt__ = lambda self, other: self.val < other.val
        
        pq = []
        # 将lists中所有的linked lists的第一个node都加入pq中
        for node in lists:
            if node != None:
                heapq.heappush(pq, node)
        
        # 由于要添加node，用dummy head化简情况！注意dummy head所在的linked list就是最终结果！！！
        dummy_head = ListNode()
        # curr始终指向最终结果(linked list)中的最后一个node，方便添加新的node！
        curr = dummy_head
        
        # 重复操作直到pq中没有node为止！
        while len(pq) != 0:
            # 取出pq中val最小的node
            node = heapq.heappop(pq)
            # 将取出来的node放入最终结果(linked list)中
            curr.next = node
            # 将取出来的node的下一个node放入pq中继续参与排序！
            if node.next != None:
                heapq.heappush(pq, node.next)
            # 移动pointer，使得curr始终指向最终结果(linked list)中的最后一个node，方便添加node！
            curr = curr.next
        
        return dummy_head.next